In [93]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import sklearn
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_json('traceGroundTruthJSON-0.json',lines=True)

In [3]:
df.head()

,type,sendTime,sender,senderPseudo,messageID,pos,pos_noise,spd,spd_noise,acl,acl_noise,hed,hed_noise
0,4,240.602763,9,1092,4184,"[266.1342377193001, 41.31755030776682, 0.0]","[4.534752768029631, 4.549525298598843, 0.0]","[-0.08087719292206601, 0.7826628559043071, 0.0]","[5.8479561322000003e-05, -0.000565907804593, 0.0]","[-0.264399876770306, 2.55929226383305, 0.0]","[0.000781535042957, 0.000157147677041, 0.0]","[-0.10279023863398801, 0.9947030546055271, 0.0]","[0.34491692230051, 0.31638175763944104, 0.0]"
1,4,241.602763,9,1092,4194,"[266.06130476311273, 43.37327132906824, 0.0]","[4.476469413385379, 4.331702683063567, 0.0]","[-0.307226018249808, 2.973074697703729, 0.0]","[0.00022214435466300002, -0.002149695059419, 0.0]","[-0.16867421025649001, 1.632928207206669, 0.0]","[0.00016366479334100001, 0.0015837872548250002...","[-0.102682182286077, 0.994714214958733, 0.0]","[0.352813203036476, 0.25414594164242404, 0.0]"
2,4,242.602763,9,1092,4204,"[265.50620000532376, 47.479615917740944, 0.0]","[4.642016156453815, 4.406704772182185, 0.0]","[-0.522133162849577, 5.052698244401898, 0.0]","[0.00037753164498000003, -0.003653380763227, 0.0]","[-0.250340680719852, 2.423187784894591, 0.0]","[0.00015538729031600002, 0.0015036857038070001...","[-0.10254626897553601, 0.994728235609705, 0.0]","[0.360477346594304, 0.20642682168206902, 0.0]"
3,4,243.602763,9,1092,4214,"[264.8798230282204, 53.81269421777557, 0.0]","[4.46289078603543, 4.600132676334774, 0.0]","[-0.7501579494382471, 7.25934146781901, 0.0]","[0.0005424091366300001, -0.005248903322175, 0.0]","[-0.15859373985413902, 1.535395525905717, 0.0]","[0.000164877491649, 0.001595522558947, 0.0]","[-0.10231333056359201, 0.9947522216054531, 0.0]","[0.368791704558564, 0.165551455629279, 0.0]"
4,4,244.602763,9,1092,4224,"[263.74132084628656, 62.04745136461824, 0.0]","[4.359550875382791, 4.449767087397457, 0.0]","[-1.196011717133152, 9.262387108898814, 0.0]","[0.000864806049117, -0.006697196551177001, 0.0]","[-0.323323397706463, 2.504311894356131, 0.0]","[0.00032239691248600003, 0.001448293229002, 0.0]","[-0.12733117487307702, 0.9918602582548821, 0.0]","[0.38560754428035204, 0.135501165937049, 0.0]"


In [4]:
df1 = df[['pos','spd']]

In [5]:
df1.head()

,pos,spd
0,"[266.1342377193001, 41.31755030776682, 0.0]","[-0.08087719292206601, 0.7826628559043071, 0.0]"
1,"[266.06130476311273, 43.37327132906824, 0.0]","[-0.307226018249808, 2.973074697703729, 0.0]"
2,"[265.50620000532376, 47.479615917740944, 0.0]","[-0.522133162849577, 5.052698244401898, 0.0]"
3,"[264.8798230282204, 53.81269421777557, 0.0]","[-0.7501579494382471, 7.25934146781901, 0.0]"
4,"[263.74132084628656, 62.04745136461824, 0.0]","[-1.196011717133152, 9.262387108898814, 0.0]"


In [6]:
split = pd.DataFrame(df1['pos'].to_list(), columns = ['posX', 'posY','posZ'])

In [7]:
split.head()

,posX,posY,posZ
0,266.134238,41.317550,0.0
1,266.061305,43.373271,0.0
2,265.506200,47.479616,0.0
3,264.879823,53.812694,0.0
4,263.741321,62.047451,0.0


In [8]:
split = split.drop('posZ',axis=1)

In [9]:
df1 = pd.concat([df1, split], axis=1) 

In [10]:
df1.head()

,pos,spd,posX,posY
0,"[266.1342377193001, 41.31755030776682, 0.0]","[-0.08087719292206601, 0.7826628559043071, 0.0]",266.134238,41.317550
1,"[266.06130476311273, 43.37327132906824, 0.0]","[-0.307226018249808, 2.973074697703729, 0.0]",266.061305,43.373271
2,"[265.50620000532376, 47.479615917740944, 0.0]","[-0.522133162849577, 5.052698244401898, 0.0]",265.506200,47.479616
3,"[264.8798230282204, 53.81269421777557, 0.0]","[-0.7501579494382471, 7.25934146781901, 0.0]",264.879823,53.812694
4,"[263.74132084628656, 62.04745136461824, 0.0]","[-1.196011717133152, 9.262387108898814, 0.0]",263.741321,62.047451


In [11]:
df1 = df1.drop('pos',axis=1)

In [12]:
split = pd.DataFrame(df1['spd'].to_list(), columns = ['spdX', 'spdY','spdZ'])

In [13]:
split = split.drop('spdZ',axis=1)

In [14]:
split.head()

,spdX,spdY
0,-0.080877,0.782663
1,-0.307226,2.973075
2,-0.522133,5.052698
3,-0.750158,7.259341
4,-1.196012,9.262387


In [15]:
df1 = pd.concat([df1, split], axis=1) 

In [16]:
df1.head()

,spd,posX,posY,spdX,spdY
0,"[-0.08087719292206601, 0.7826628559043071, 0.0]",266.134238,41.317550,-0.080877,0.782663
1,"[-0.307226018249808, 2.973074697703729, 0.0]",266.061305,43.373271,-0.307226,2.973075
2,"[-0.522133162849577, 5.052698244401898, 0.0]",265.506200,47.479616,-0.522133,5.052698
3,"[-0.7501579494382471, 7.25934146781901, 0.0]",264.879823,53.812694,-0.750158,7.259341
4,"[-1.196011717133152, 9.262387108898814, 0.0]",263.741321,62.047451,-1.196012,9.262387


In [17]:
df1 = df1.drop('spd',axis=1)

In [18]:
df1.head()

,posX,posY,spdX,spdY
0,266.134238,41.317550,-0.080877,0.782663
1,266.061305,43.373271,-0.307226,2.973075
2,265.506200,47.479616,-0.522133,5.052698
3,264.879823,53.812694,-0.750158,7.259341
4,263.741321,62.047451,-1.196012,9.262387


In [19]:
df1

,posX,posY,spdX,spdY
0,266.134238,41.317550,-0.080877,0.782663
1,266.061305,43.373271,-0.307226,2.973075
2,265.506200,47.479616,-0.522133,5.052698
3,264.879823,53.812694,-0.750158,7.259341
4,263.741321,62.047451,-1.196012,9.262387
...,...,...,...,...
5021,139.526900,876.928567,0.001691,0.001691
5022,139.749557,876.918451,0.001691,0.001691
5023,139.412345,878.207028,-0.296062,2.253724
5024,138.107917,881.136414,-1.281437,3.882940


In [77]:
input_comb = []

for i in range(len(df)):
    if i%20==0 and len(df)-i>20:
        curr = []
        for j in range(20):
            curr.append(np.array(df1.iloc[i+j]))
        input_comb.append(curr)

input_comb = np.array(input_comb)
input_comb = torch.from_numpy(input_comb)

In [78]:
input_comb

tensor([[[ 2.6613e+02,  4.1318e+01, -8.0877e-02,  7.8266e-01],
         [ 2.6606e+02,  4.3373e+01, -3.0723e-01,  2.9731e+00],
         [ 2.6551e+02,  4.7480e+01, -5.2213e-01,  5.0527e+00],
         ...,
         [ 2.3548e+02,  2.4216e+02, -2.1530e+00,  1.4321e+01],
         [ 2.3326e+02,  2.5648e+02, -2.1514e+00,  1.4311e+01],
         [ 2.3106e+02,  2.7082e+02, -2.1599e+00,  1.4369e+01]],

        [[ 2.2909e+02,  2.8517e+02, -2.1589e+00,  1.4361e+01],
         [ 2.2709e+02,  2.9952e+02, -2.1595e+00,  1.4365e+01],
         [ 2.2478e+02,  3.1385e+02, -2.1601e+00,  1.4370e+01],
         ...,
         [ 1.8989e+02,  5.2693e+02, -1.6520e+00,  1.0965e+01],
         [ 1.8857e+02,  5.3545e+02, -9.8135e-01,  6.5149e+00],
         [ 1.8817e+02,  5.3952e+02, -3.1199e-01,  2.0737e+00]],

        [[ 1.8816e+02,  5.3990e+02,  4.8300e-04,  4.8300e-04],
         [ 1.8822e+02,  5.3989e+02,  4.8300e-04,  4.8300e-04],
         [ 1.8820e+02,  5.3990e+02,  4.8300e-04,  4.8300e-04],
         ...,
         

In [84]:
class Dataset(torch.utils.data.Dataset):
#   'Characterizes a dataset for PyTorch'
    def __init__(self, list_IDs, labels):
#         'Initialization'
        self.labels = labels
        self.list_IDs = list_IDs
        
    def __len__(self):
#         'Denotes the total number of samples'
        return len(self.list_IDs)

    def __getitem__(self, index):
#         'Generates one sample of data'
        # Select sample
        ID = self.list_IDs[index]

        # Load data and get label
        X = torch.load('data/' + ID + '.pt')
        y = self.labels[ID]

        return X, y

In [88]:
labels = torch.zeros(len(input_comb))
labels.shape

torch.Size([251])

In [95]:
X_train, X_rem, y_train, y_rem = train_test_split(input_comb,labels, train_size=0.85)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

In [102]:
training_data = Dataset(X_train,y_train)
validation_data = Dataset(X_valid,y_valid)
test_data = Dataset(X_test,y_test)

In [103]:
train_dataloader = DataLoader(training_data, batch_size=32, shuffle=True)
val_dataloader = DataLoader(training_data, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=32, shuffle=True)

In [25]:
class CNN_LSTM(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv1d(in_channels=20, out_channels=20, kernel_size=3)
        
        self.lstm1 = nn.LSTM(input_size=2,hidden_size=2,num_layers=256)
        self.lstm2 = nn.LSTM(input_size=2,hidden_size=2,num_layers=256)
        self.lstm3 = nn.LSTM(input_size=2,hidden_size=2,num_layers=256)
        self.lstm4 = nn.LSTM(input_size=2,hidden_size=2,num_layers=256)
        
        self.lin1 = nn.Linear(2,1)
        self.lin2 = nn.Linear(4,2)
        
    def forward(self,x):
        h0_1 = torch.randn(256,2)
        c0_1 = torch.randn(256,2)
        h0_2 = torch.randn(256,2)
        c0_2 = torch.randn(256,2)
        h0_3 = torch.randn(256,2)
        c0_3 = torch.randn(256,2)
        h0_4 = torch.randn(256,2)
        c0_4 = torch.randn(256,2)
        
        x = self.conv(x)
        
        o1, _ = self.lstm1(x,(h0_1,c0_1))
        o2, _ = self.lstm2(x,(h0_2,c0_2))
        o3, _ = self.lstm3(x,(h0_3,c0_3))
        o4, _ = self.lstm4(x,(h0_4,c0_4))
        
        out1 = self.lin1(o1)
        out2 = self.lin1(o2)
        out3 = self.lin1(o3)
        out4 = self.lin1(o4)
        
        out_comb = torch.stack((out1,out2,out3,out4),dim=1)
        out_comb = out_comb.squeeze()
        
        out_final = self.lin2(out_comb)
        
        return out_final

In [26]:
input = torch.randn(20,4)
print(input)

tensor([[ 1.2233e-01, -1.5560e-01, -4.9400e-01,  6.7226e-01],
        [-5.4694e-01, -2.3209e-01, -8.1113e-01, -6.0431e-01],
        [-1.4862e-01, -9.5942e-01, -2.6626e-01,  9.8797e-01],
        [-9.4739e-01,  6.5095e-04,  2.0512e+00,  9.8368e-03],
        [ 1.8265e+00,  1.6785e+00, -2.2631e-01,  4.2477e-01],
        [-1.0079e+00,  1.8331e+00,  6.0375e-01, -7.6286e-01],
        [ 2.5227e-01, -2.3700e+00,  3.7274e-01,  2.4495e-01],
        [ 7.0091e-01, -1.4010e-02,  1.9317e+00,  3.3166e-01],
        [-3.0616e-01,  3.5829e-01, -1.3450e-01, -3.0135e-01],
        [-8.4556e-01, -1.9253e-01, -1.8497e+00,  1.3777e+00],
        [ 1.8814e-01,  3.7021e-01, -2.6443e-01,  1.2608e+00],
        [ 6.4532e-03, -3.3096e-01, -3.8760e-01, -1.9621e+00],
        [ 1.7017e+00, -1.2875e+00,  2.4694e-02, -6.7430e-01],
        [-6.1282e-01, -1.4654e+00, -3.8547e-01,  3.8815e-01],
        [-1.0066e+00,  1.8358e+00, -1.5350e+00,  2.6878e-01],
        [ 4.3262e-01, -1.1542e+00,  1.6873e+00, -1.9876e-02],
        

In [27]:
model = CNN_LSTM()

In [28]:
output = model(input)

In [29]:
output

tensor([[-2.9574e-01, -9.7269e-03],
        [-2.8415e-01,  2.5794e-04],
        [-2.6729e-01, -6.3583e-04],
        [-2.5451e-01,  2.6584e-04],
        [-2.4663e-01,  1.8778e-03],
        [-2.4208e-01,  3.3137e-03],
        [-2.3954e-01,  4.2740e-03],
        [-2.3816e-01,  4.7990e-03],
        [-2.3743e-01,  5.0199e-03],
        [-2.3705e-01,  5.0571e-03],
        [-2.3686e-01,  4.9961e-03],
        [-2.3676e-01,  4.8906e-03],
        [-2.3671e-01,  4.7718e-03],
        [-2.3669e-01,  4.6564e-03],
        [-2.3668e-01,  4.5521e-03],
        [-2.3667e-01,  4.4619e-03],
        [-2.3666e-01,  4.3858e-03],
        [-2.3666e-01,  4.3228e-03],
        [-2.3666e-01,  4.2714e-03],
        [-2.3666e-01,  4.2297e-03]], grad_fn=<AddmmBackward0>)

In [83]:
def thresholding_algorithm(X, Y, iterations=100, starting_threshold=0, precision=0.1):
    device = X.device # get the device of the input tensors
    N = iterations
    threshold = torch.linspace(starting_threshold, starting_threshold + (N-1)*precision, N, device=device)
    st = starting_threshold
    pre = precision
    m = int(torch.ceil(torch.log10(torch.Tensor([N]).to(device))))
    for l in range(m):
        Accuracy = torch.zeros(N, device=device)
        for i in range(N):
            for j in range(len(X)):
                mae = torch.mean(torch.abs(X[j]-X[j]))
                if (mae < threshold[i] and Y[j] == 0) or (mae > threshold[i] and Y[j] != 0):
                    Accuracy[i] += 1/len(X)
        threshold_max = st + (torch.argmax(Accuracy))/pre
        en = st + ((torch.argmax(Accuracy) + 1)/pre)
        st = st + ((torch.argmax(Accuracy) - 1)/pre)
        pre *= 10
        N = int((en - st) * pre)
        threshold = torch.linspace(st, en, N, device=device)
    return threshold_max